## Imports / Definitions

In [ ]:
import yaml
import os
from importlib import reload
from altair import *
from altair_widgets import interact_with
import pandas as pd

%load_ext yamlmagic

In [ ]:
# Define Enumerations here
'''
class ClientType:
    CLOSED = 0
    OPEN = 1
    
class Benchmark:
    EXAMPLE = 'example'
'''

## Experimental Settings

In [ ]:
username = "" # Username of user on server
exp_name = "" # Name of Experiment on TestBed
proj_name = "" # Name of Project on TestBed
default_node = "" # Default node to run the experiment from
ds = ""
host_ip = ""
host_servers_ip = ""
default_ip = "{}.{}.{}.{}".format(default_node, exp_name, proj_name, host_servers_ip)

# change into the correct directory
proj_path = "~/DSEF/{}".format(ds)
%cd '$proj_path'

In [ ]:
# experiment related variables
'''
duration = 10              # duration of each test
name = "dsef"              # name of the experiment
clients = [1, 2, 4, 8] # clients to run tests with
servers = [2]              # number of servers to run tests with
benchs = [Benchmark.EXAMPLE]   # benchmark to run
num_cores = 2              # number of cores each server has
data_centers = []
client = ClientType.CLOSED # is client open or closed
'''

## Hosts

In [ ]:
%%yaml hosts
host:
  node-1: node-1
  node-2: node-2
  node-3: node-3
  node-4: node-4
  node-5: node-5

In [ ]:
# write hosts to file
with open('hosts.yml', 'w') as f:
    yaml.dump(hosts, f, default_flow_style=False)

# Experiment
Launch Instances of the Distributed System
    
Setup Benchmark on Clients
    
Start the Experiment
    
Kill all Instances of the Distributed System 

## Initialization

In [ ]:
# import <name of experiment generation>
import gen_experiments
reload(gen_experiments)
experiments = gen_experiments.main(duration, name, clients, servers, benchs, num_cores, data_centers, client)

# Loop

In [ ]:
import dsef_client as dsef
from time import sleep
reload(dsef)

exp = dsef.Experiment(default_ip, username, ds)

files = ['run.py']
exp.transfer_files(files)

exp.init_experiment()

print("[+] Running {} Experiments".format(len(experiments)))
for e in experiments:
    exp.transfer_files(e[2])
    if not exp.run_experiment(e[0], e[1], e[2], duration):
        break
    
data = exp.end()

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
interact_with(df.transpose())